In [1]:
import sys                                                                                              #import libraries
import math
from collections import defaultdict
from decimal import Decimal
# from preprocess import preprocessing
import operator
import string
import random

In [2]:
def preprocessing(numberOfSentences, sourceFile, targetFile):

    indices=[]
    trainingSource=[]
    trainingTarget=[]
    testingSource=[]
    testingTarget=[]
    sentencePair={}
    print(sourceFile)
    print(targetFile)
    for i in range(numberOfSentences):                                                  #create random numbers                           
        indices.append(random.randint(0,167689))
    train_lines = 0
    test_lines = 0
    with open(sourceFile,'r', encoding="utf8") as  fSource:                                              #read from source language corpus
        for index,line in enumerate(fSource):
            if len(line)>0:
                if index in indices:
                    trainingSource.append(line.strip())   
                    train_lines += 1                              #create training and testing for source language
                else:
                    testingSource.append(line.strip())
                    test_lines += 1
    print(train_lines, test_lines)
                
    with open(targetFile,'r', encoding="utf8") as  fTarget:                                              #read from source language corpus
        for index,line in enumerate(fTarget):
            if len(line)>0:
                if index in indices:
                    trainingTarget.append(line.strip())                                 #create training and testing for source language
                else:
                    testingTarget.append(line.strip())
    
    table = str.maketrans("","", string.punctuation)                                                   #create sentence pair for training set
    for i in range(len(trainingSource)):
        sentencePair[trainingSource[i].translate(table).lower()]=trainingTarget[i].translate(table).lower()
            
            
    with open('trainingSource.txt','w', encoding="utf8") as f:                                          #write into training file for source data
        f.write('\n'.join(trainingSource))

    with open('trainingTarget.txt','w', encoding="utf8") as f:                                          #write into training file for target data
        f.write('\n'.join(trainingTarget))

    testingSource=testingSource[:numberOfSentences]                                     #write into testing file for source data
    with open('testingSource.txt','w', encoding="utf8") as f:
        f.write('\n'.join(testingSource))

    testingTarget=testingTarget[:numberOfSentences]                                     #write into testing file for target data
    with open('testingTarget.txt','w', encoding="utf8") as f:
        f.write('\n'.join(testingTarget))
    print("Preprocessing done")
    return sentencePair


In [3]:
def calculatePerplexity(sentencePair, translationProbability):                                          #Perplexity Calculation
    newperplexity=0
    for pair in sentencePair:
        sourceWords=pair.split(' ')
        targetWords=sentencePair[pair].split(' ')
        ltarget=len(targetWords)
        lsource=len(sourceWords)
        probability=Decimal(0)
        for words in targetWords:
            if words!='':
                total=0
                for key in sourceWords:
                    if key!='':
                        total+=translationProbability[words][key]
                if total!=0:
                    probability+=Decimal(math.log(total,2))
        probability-=Decimal(math.log((lsource**ltarget),2))                                
        newperplexity+=Decimal(probability)
    newperplexity=-newperplexity
    newperplexity=2**newperplexity
    return newperplexity
        

In [4]:
numberOfSentences = 167689
numberOfIterations = 30
sentencePair = preprocessing(numberOfSentences, "normal_deleted.aligned", "simple_deleted.aligned")

listOfSourceWords = defaultdict(list)                                                               #create list of possible source words for a target word
for pair in sentencePair:
    sourceWords=pair.split(' ')
    targetWords=sentencePair[pair].split(' ')
    for word in targetWords:
        if word!='':
            for key in sourceWords:
                if key!='':
                    listOfSourceWords[word].append(key)
                    
for word in  listOfSourceWords:
       listOfSourceWords[word] = list(set(listOfSourceWords[word]))

normal_deleted.aligned
simple_deleted.aligned
105831 61858


In [5]:
tokenized_stores = {'normal_train': [], 'simple_train': [], 'normal_test': [], 'simple_test': []}

for key in tokenized_stores:
    if str(key) == "normal_train":
        file_name = "normal_deleted.aligned"
    elif str(key) == "simple_train":
        file_name = "simple_deleted.aligned"
    elif str(key) == "normal_test":
        file_name = "normal_aligned_test.txt"
    else: 
        file_name = "simple_aligned_test.txt"
    load = open(file_name, encoding = "utf8")
    sentences = load.read().split('\n')
        
    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)

train_size = len(tokenized_stores['normal_train'])
    # dev_size = len(tokenized_stores['normal_dev'])
test_size = len(tokenized_stores['normal_test'])


In [6]:
translationProbability=defaultdict(dict)                                                            #initialize the translation probability
for wordTarget in  listOfSourceWords:
    uniqueWordsSource = listOfSourceWords[wordTarget]
    for wordSource in uniqueWordsSource:
        translationProbability[wordTarget][wordSource]=1/float(len(uniqueWordsSource))


In [7]:
perplexity=0                                                                                        #Expectation Maximisation
iteration=0
while iteration<numberOfIterations:
    iteration+=1 
    print ("Iteration: " +str(iteration))
        
    count=defaultdict(lambda:  defaultdict(float))                                                  #initialisation
    sumTotal=defaultdict(float)
    total=defaultdict(float)
            
    for pair in sentencePair:                                                                       #E Step
        sourceWords=pair.split(' ')
        targetWords=sentencePair[pair].split(' ')
        for words in targetWords:
            if words!='':
                for key in sourceWords:
                    if key!='':
                        sumTotal[words]+=translationProbability[words][key]

        for words in targetWords:
            if words!='':
                for key in sourceWords:
                    if key!='':
                        count[words][key]+=(translationProbability[words][key]/sumTotal[words])
                        total[key]+=(translationProbability[words][key]/sumTotal[words])
          
    for key in sourceWords:                                                                         #M Step
        if key!='':
            for words in targetWords:
                if words!='':
                    translationProbability[words][key]=count[words][key]/total[key]

#     newperplexity=calculatePerplexity(sentencePair, translationProbability)
#     print (newperplexity)
#     if perplexity>=newperplexity:
#         print ("Successful")
#     else:
#         print ("Failed")

# perplexity = newperplexity


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30


In [8]:
data = {}
writing_data = []
# print (len(translationProbability))
for key in translationProbability:
    possibleWords = translationProbability[key]
    word = max(possibleWords.items(), key = operator.itemgetter(1))[0]
    writing_data.append(key + ', ' + word + ', ' + str(translationProbability[key][word]))
    data[(key, word)] = translationProbability[key][word]
        # print(key, word, data[(key,word)])
with open('wordTranslation_nice.txt','w', encoding="utf8") as f:
    f.write('\n'.join(writing_data))

In [53]:
bigrams = {}
unigrams = {}
trigrams = {}
# training on the train_set
def model(dataset_size, dataset_name):
    # global bigrams
    # global unigrams
    for index in range(dataset_size):
        token_A = ''
        token_B = ''
        # print(dataset_name, index, len(tokenized_stores[dataset_name]))
        for simple_token in tokenized_stores[dataset_name][index]:
            if simple_token not in unigrams:
                unigrams[simple_token] = 1
            else:
                unigrams[simple_token] += 1
            
            token_C = simple_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            
            if (token_A, token_B, token_C) not in trigrams:
                trigrams[(token_A, token_B, token_C)] = 1
            else:
                trigrams[((token_A, token_B, token_C))] += 1
            token_A = token_B
            token_B = token_C


In [54]:
print(test_size)
print(train_size)
model(train_size, 'simple_train')
# model(dev_size, 'simple_dev')
trigram_count = len(trigrams)
bigram_count = len(bigrams)
unigram_count = len(unigrams)
x = 0
for i in trigrams.keys():
    print(trigrams[i], i)
    x += 1
    if x == 10:
        break
print("Number of Unique Trigrams:", trigram_count)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)


33538
167690
8703 ('', '', 'It')
3484 ('', 'It', '')
6 ('', '', 'Cherokee')
2 ('', 'Cherokee', 'of')
2 ('Cherokee', 'of', 'the')
1135 ('of', 'the', '')
1 ('', '', 'Skateboard')
1 ('', 'Skateboard', 'normally')
1 ('Skateboard', 'normally', '')
29712 ('', '', 'The')
Number of Unique Trigrams: 553699
Number of Unique Bigrams: 317099
Number of Unique Unigrams: 59914


In [55]:
from itertools import permutations
import nltk

In [65]:
computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(data.items(), key = lambda k:(k[1], k[0]), reverse = True)
# print("Sorting done")
def find_translation(normal_token):
#     print("Find translation called")
    for element in sorted_t:
        if element[0][0].lower() == normal_token:
#             print(element[0], element[1])
            if element[1] >= 0.01:
                return element[0][1]
            else:
                return normal_token
    return normal_token

# def get_prob(seq):
# # bigram language model with laplace smoothing and backoff
# #         print("Get prob called")
#     if len(seq) < 2:
#         return 1
#     score = 0
#     token_A = ''
#     for simple_token in seq:
#         token_B = simple_token
#         if(token_A, token_B) not in bigrams:
#             if token_B not in unigrams:
#                 continue
#             else:
#                 score += unigrams[token_B] / unigram_count
#         else:
#             if (token_A, token_B) not in bigrams.keys():
#                 bigrams[(token_A, token_B)] = 0
#             if token_A not in unigrams.keys():
#                 unigrams[token_A] = 0
#             score += (bigrams[(token_A, token_B)] + 1)/ (unigrams[token_A] + unigram_count)
#         token_A = token_B
# #     print("returning from get prob")
#     return score


def get_prob(seq):
    if len(seq) < 3:
        return 1
    score = 0
    token_A = ''
    token_B = ''
    for simple_token in seq:
        token_C = simple_token
        if(token_A, token_B, token_C) not in trigrams:
            if (token_B, token_C) not in bigrams:
                if token_C not in unigrams:
                    continue
                else:
                    score += unigrams[token_C]/ unigram_count
            else:
                if (token_B, token_C) not in bigrams.keys():
                    bigrams[(token_B,token_C)] = 0
                if token_B not in unigrams.keys():
                    unigrams[token_A] = 0
                score += (bigrams[(token_B, token_C)] + 1)/ (unigrams[token_B] + unigram_count)
        else:
            if (token_A, token_B, token_C) not in trigrams.keys():
                trigrams[(token_A, token_B, token_C)] = 0
            if (token_A, token_B) not in bigrams.keys():
                bigrams[(token_A, token_B)] = 0
            if token_A not in unigrams.keys():
                unigrams[token_A] = 0
            score += (trigrams[(token_A, token_B, token_C)] + 1)/ (bigrams[(token_A, token_B)] + bigram_count)

        token_A = token_B
        token_B = token_C
    return score
        
count = 0
for index in range(test_size):
    if len(tokenized_stores['normal_test'][index]) > 8 or len(tokenized_stores['normal_test'][index]) < 2:
        continue

    translated_words = []
    for normal_token in tokenized_stores['normal_test'][index]:
        translation = find_translation(normal_token)
        if translation != "":
            translated_words.append(translation)
    perm = permutations(translated_words)
#     print("Perm done")
    best_seq = translated_words
    best_prob = -1

    for seq in perm:
        prob = get_prob(seq)
        if prob > best_prob:
            best_prob = prob
            best_seq = seq
#     print("Best seq found")
    BLEU_scores = []

# Collecting BLEU_scores with various kinds of Smoothing
#     print("BLEU scores started")
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method2))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method3))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method5))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['simple_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))

    for key in total_BLEU.keys():
        if key == 7:
            consider = 5
        else: 
            consider = key - 1
        total_BLEU[key] += BLEU_scores[consider]
        
    if BLEU_scores[0] == 0:
        null_BLEU_count += 1
        
    count += 1
    print("Sentence Index: ", str(count))
    print("Normal Sentence:", str(tokenized_stores['normal_test'][index]))
    print("Simple Sentence:", str(tokenized_stores['simple_test'][index]))
    print("Translated Sentence:", str(best_seq))
    print("Translation BLEU Scores", str(BLEU_scores))
#     x = 0
#     for i in range(min(len(seq), len(tokenized_stores['simple_test'][index]))):
#         if tokenized_stores['simple_test'][index][i] == best_seq[i]:
#             x += 1
#             if x > (min(len(seq), len(tokenized_stores['simple_test'][index]))+1)/2:
#                 print("Normal Sentence:", str(tokenized_stores['normal_test'][index]))
#                 print("Simple Sentence:", str(tokenized_stores['simple_test'][index]))
#                 print("Translated Sentence:", str(best_seq))
#     print(x, min(len(seq), len(tokenized_stores['simple_test'][index])))
    
    computed_sentences.append([tokenized_stores['normal_test'][index], tokenized_stores['simple_test'][index], best_seq, BLEU_scores])
    if count == 1000:
        break
    
tested = count

Sentence Index:  1
Normal Sentence: ['His', 'program', 'was', 'called', 'Piper', "'s", 'Tune', '.']
Simple Sentence: ['The', 'name', 'of', 'his', 'program', 'was', 'called', 'Piper', "'s", 'Tune', '.']
Translated Sentence: ('program', 'was', 'Piper', 'Tune', "'s", 'His', '.', 'called')
Translation BLEU Scores [0.055218773861664086, 0.1853634560866775, 0.10382794589030601, 0.1850139215121847, 0.14146919011470205, 0.2558845545740301]
Sentence Index:  2
Normal Sentence: ['There', 'were', 'three', 'premaxillary', 'teeth', '.']
Simple Sentence: ['There', 'were', 'three', 'premaxillary', 'teeth', '.']
Translated Sentence: ('three', 'There', 'premaxillary', 'formula', 'were', '.')
Translation BLEU Scores [0.06104735835807843, 0.29071536848410967, 0.12137294292683086, 0.2983682542622611, 0.17106674642655578, 0.3801955718365812]
Sentence Index:  3
Normal Sentence: ['=', '-LRB-', 'n', '1', '-RRB-', '!']
Simple Sentence: ['is', 'the', 'number', 'we', 'get', 'when', 'we', 'multiply', 'every', 'num

Sentence Index:  21
Normal Sentence: ['The', 'number', 'of', 'Cornish', 'speakers', 'is', 'growing', '.']
Simple Sentence: ['No', 'one', 'knows', 'how', 'many', 'Cornish', 'speakers', 'there', 'are', 'now', '.']
Translated Sentence: ('number', 'picardy', 'Cornish', 'is', 'growing', 'The', 'speakers', '.')
Translation BLEU Scores [0.025124218547395095, 0.13107175678306446, 0.04995138898472386, 0.17925515383388427, 0.07715684863466059, 0.22377842029956965]
Sentence Index:  22
Normal Sentence: ['Its', 'county', 'seat', 'is', 'Sebring', ',', 'Florida', '.']
Simple Sentence: ['The', 'county', 'seat', 'is', 'Sebring']
Translated Sentence: ('county', 'Its', 'Sebring', 'is', 'seat', 'Florida', '.', ',')
Translation BLEU Scores [0.039281465090051315, 0.20164945583740668, 0.07809849842300641, 0.2802636449867835, 0.1283000598199168, 0.34539629931163957]
Sentence Index:  23
Normal Sentence: ['His', 'father', 'was', 'a', 'soldier', '.']
Simple Sentence: ['His', 'father', 'was', 'a', 'soldier', '.']

Sentence Index:  41
Normal Sentence: ['The', 'Indian', 'Ocean', 'has', 'asymmetric', 'ocean', 'circulation', '.']
Simple Sentence: ['The', 'Indian', 'Ocean', 'is', 'the', 'ocean', 'surrounded', 'by', 'Asia', 'to', 'the', 'north', ',', 'Australia', 'and', 'the', 'Pacific', 'Ocean', 'to', 'the', 'east', ',', 'the', 'Southern', 'Ocean', 'to', 'the', 'south', ',', 'and', 'Africa', 'and', 'the', 'Atlantic', 'Ocean', 'to', 'the', 'west', '.']
Translated Sentence: ('document', 'Indian', 'has', 'ocean', 'The', 'Ocean', 'core', '.')
Translation BLEU Scores [0.0008620333407051443, 0.004380273704134083, 0.0017138747077101882, 0.006150397028530028, 0.0029956306395582484, 0.007570927537206954]
Sentence Index:  42
Normal Sentence: ['See', 'also', 'Sophie', 'Germain', 'primes', '.']
Simple Sentence: ['The', 'Sophie', 'Germain', 'prime', 'and', 'Crater', 'Germain', 'on', 'Venus', 'are', 'named', 'after', 'her', '.']
Translated Sentence: ('Germain', 'See', 'generalized', '.', 'Sophie', 'also')
Translat

Sentence Index:  61
Normal Sentence: ['The', 'ants', 'will', 'eventually', 'die', 'of', 'exhaustion', '.']
Simple Sentence: ['The', 'ants', 'will', 'die', 'of', 'exhaustion', '.']
Translated Sentence: ('species', 'The', 'picardy', 'exhaustion', 'will', 'die', 'eventually', '.')
Translation BLEU Scores [0.07386099955930606, 0.25098621243978964, 0.13888095170058956, 0.24747585322180027, 0.17369121414402194, 0.33433107018058583]
Sentence Index:  62
Normal Sentence: ['The', 'county', 'seat', 'is', 'Colusa', '.']
Simple Sentence: ['Its', 'county', 'seat', 'is', 'Colusa', '.']
Translated Sentence: ('county', 'is', 'seat', 'The', 'Colusa', '.')
Translation BLEU Scores [0.10855926040543844, 0.345720784641941, 0.20412414523193154, 0.2784105955548633, 0.21206804158885403, 0.36743157422771194]
Sentence Index:  63
Normal Sentence: ['The', 'organ', 'has', 'four', 'manuals', 'and', 'pedalboard', '.']
Simple Sentence: ['The', 'organ', 'has', 'four', 'manuals', 'and', 'pedalboard', '.']
Translated Sen

Sentence Index:  82
Normal Sentence: ['â', 'It', 'is', 'bottled', 'in', 'PET', 'bottles', '.']
Simple Sentence: ['â', 'It', 'is', 'bottled', 'in', 'PET', 'bottles', '.']
Translated Sentence: ('It', 'bottled', 'PET', 'he', 'â', 'is', '.', 'in')
Translation BLEU Scores [0.045180100180492254, 0.22679164443904004, 0.08982602799077107, 0.32234896352325065, 0.17641258225238565, 0.40261574351230733]
Sentence Index:  83
Normal Sentence: ['Sometimes', 'it', 'is', 'simply', 'called', 'as', 'C2']
Simple Sentence: ['Sometimes', 'it', 'is', 'simply', 'called', 'as', 'C2']
Translated Sentence: ('northern', 'is', 'Sometimes', 'simply', 'C2', 'as', 'called')
Translation BLEU Scores [0.051697315395717064, 0.25406637407730737, 0.10278340422512992, 0.3116670168631123, 0.17412150975560142, 0.39268056506659094]
Sentence Index:  84
Normal Sentence: ['The', 'term', 'C2', 'is', 'actually', 'an', 'abbreviation', '.']
Simple Sentence: ['The', 'term', 'C2', 'is', 'actually', 'an', 'abbreviation', '.']
Translated

...

Sentence Index:  899
Normal Sentence: ['Elijah', 'was', 'played', 'at', 'every', 'successive', 'festival', '.']
Simple Sentence: ['Elijah', 'was', 'played', 'at', 'every', 'festival', 'after', 'that', '.']
Translated Sentence: ('played', 'Elijah', 'at', 'every', 'was', 'successive', 'festival', '.')
Translation BLEU Scores [0.07090230911670939, 0.23801138894037596, 0.13331772148563245, 0.23756257766271607, 0.1816500357855076, 0.32856227181087616]
Sentence Index:  900
Normal Sentence: ["'s", 'career', 'stats', 'at', 'Hockey-Reference', '.']
Simple Sentence: ["'s", 'career', 'stats', 'at', 'Hockey-Reference', '.']
Translated Sentence: ('stats', 'at', 'career', 'Hockey-Reference', '.', "'s")
Translation BLEU Scores [0.13512001548070346, 0.3976353643835253, 0.25406637407730737, 0.346528189680633, 0.2736118901182546, 0.4316433028181803]
Sentence Index:  901
Normal Sentence: ['He', 'began', 'writing', 'in', '1967', '.']
Simple Sentence: ['First', 'literary', 'experiences', ':', '1967', '.']


Sentence Index:  920
Normal Sentence: ['James', 'Mease', 'published', 'another', 'recipe', 'in', '1812', '.']
Simple Sentence: ['James', 'Mease', 'published', 'another', 'recipe', 'in', '1812', '.']
Translated Sentence: ('Mease', 'another', 'James', 'published', '21', 'robert', '.', 'in')
Translation BLEU Scores [0.043472087194499145, 0.21934566882541542, 0.08643019616048525, 0.310162708656183, 0.16037507477489607, 0.3838425690033344]
Sentence Index:  921
Normal Sentence: ['Welcome', 'to', 'Wikipedia', '.']
Simple Sentence: ['Hello', 'and', 'welcome', 'to', 'Wikipedia', '.']
Translated Sentence: ('.', 'Welcome', 'Wikipedia', 'to')
Translation BLEU Scores [0.0641328089975786, 0.25916266987614406, 0.12750736437345603, 0.15616919926099557, 0.09727239990468063, 0.20683085886733632]
Sentence Index:  922
Normal Sentence: ['It', 'grows', 'at', '1,600-4', ',900', 'm', 'altitude', '.']
Simple Sentence: ['It', 'grows', 'at', '1,600-4', ',900', 'm', 'altitude', '.']
Translated Sentence: ('grows',

Sentence Index:  941
Normal Sentence: ['There', 'are', '54', 'provinces', ',', 'in', 'total', '.']
Simple Sentence: ['There', 'are', '54', 'provinces', ',', 'in', 'total', '.']
Translated Sentence: ('.', ',', 'total', 'in', 'provinces', 'the', 'There', 'are')
Translation BLEU Scores [0.08034284189446518, 0.26970223719007375, 0.1510687698678384, 0.2691936673850745, 0.20583645704988157, 0.3723098300386164]
Sentence Index:  942
Normal Sentence: ['He', 'became', 'a', 'French', 'subject', 'in', '1661', '.']
Simple Sentence: ['He', 'took', 'on', 'French', 'nationality', 'in', '1661', '.']
Translated Sentence: ('link', 'He', 'French', 'in', 'subject', 'became', '.', 'a')
Translation BLEU Scores [0.039281465090051315, 0.20164945583740668, 0.07809849842300641, 0.2802636449867835, 0.1283000598199168, 0.34539629931163957]
Sentence Index:  943
Normal Sentence: ['It', 'has', 'about', '200', 'houses', '.']
Simple Sentence: ['It', 'has', 'about', '200', 'houses', ',', 'no', 'shops', 'and', 'one', 'po

Sentence Index:  962
Normal Sentence: ['Castro', 'has', 'remained', 'active', 'as', 'an', 'athlete', '.']
Simple Sentence: ['Castro', 'is', 'still', 'active', 'as', 'a', 'sportsman', '.']
Translated Sentence: ('remained', 'an', '.', 'has', 'active', 'as', 'Castro', 'she')
Translation BLEU Scores [0.06985342056580097, 0.23980296761827097, 0.13134549472120793, 0.23404821161541917, 0.1576071866498511, 0.31483154198915664]
Sentence Index:  963
Normal Sentence: ['Together', 'they', 'have', 'completed', 'several', 'marathons', '.']
Simple Sentence: ['Together', 'they', 'have', 'completed', 'several', 'marathons', '.']
Translated Sentence: ('they', 'Together', 'completed', 'several', 'marathons', '.', 'have')
Translation BLEU Scores [0.4728708045015879, 0.5814307369682193, 0.4728708045015879, 0.4728708045015879, 0.5371530186728025, 0.5371530186728025]
Sentence Index:  964
Normal Sentence: ['All', 'fields', 'must', 'be', 'in', 'lowercase', '.']
Simple Sentence: ['All', 'fields', 'must', 'be', 

Sentence Index:  983
Normal Sentence: ['He', 'has', 'played', 'for', 'Japan', 'national', 'team', '.']
Simple Sentence: ['He', 'has', 'played', 'for', 'Japan', 'national', 'team', '.']
Translated Sentence: ('played', 'has', 'team', 'Japan', '.', 'for', 'national', 'He')
Translation BLEU Scores [0.046713797772820025, 0.23356898886410005, 0.09287528999566796, 0.33329152069487966, 0.19245008972987526, 0.4211497836519418]
Sentence Index:  984
Normal Sentence: ['Its', 'capital', 'is', 'Sarnen', '.']
Simple Sentence: ['Its', 'capital', 'is', 'Sarnen', '.']
Translated Sentence: ('.', 'capital', 'Its', 'Sarnen', 'is')
Translation BLEU Scores [0.08034284189446518, 0.35930411196308426, 0.1597357760615681, 0.2972635645416151, 0.19245008972987526, 0.3930776032102183]
Sentence Index:  985
Normal Sentence: ['The', 'economic', 'center', 'is', 'Tapachula', '.']
Simple Sentence: ['The', 'economic', 'center', 'of', 'Soconusco', 'is', 'Tapachula', '.']
Translated Sentence: ('center', 'Tapachula', '.', 'e

In [1]:
tested = count
import statistics
# tested = 201
print("Number of Samples Tested Upon: " + str(tested))
print()
print("Average BLEU Score using Various Smoothing Functions")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / tested))
print()
print("Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / (tested - null_BLEU_count)))


NameError: name 'count' is not defined